In [1]:
import uproot
import numpy as np
import matplotlib.pyplot as plt
import awkward as ak
import sys
import coffea.hist as hist
sys.path.append("/uscms/home/sbrightt/nobackup/iDM/iDMe_analysis/CMSSW_10_6_26/src/iDMeAnalysis/python_analysis/analysisTools/")
from analysisTools import Analyzer
from analysisTools import loadNano
import analysisSubroutines as routines
import importlib
import coffea.hist as hist
import coffea.util as util
import time

In [2]:
location = "/uscms/home/sbrightt/nobackup/iDM/iDMe_analysis/CMSSW_10_6_26/src/iDMeAnalysis/python_analysis/condor/submissions/"
subsamples = ["signal_2018_aEM_histoList_{0}".format(k) for k in range(7)]
jsons = [location+s+'/samples.json' for s in subsamples]
histos = "/uscms_data/d3/sbrightt/iDM/iDMe_analysis/CMSSW_10_6_26/src/iDMeAnalysis/python_analysis/configs/histo_configs/histoList.json"

In [3]:
for i, js in enumerate(jsons):
    az = Analyzer(js,histos)
    s = subsamples[i]
    h_out = az.process()
    out = "output_"+s+".coffea"
    util.save(h_out,out)

Preprocessing:   0%|          | 0/26 [00:00<?, ?file/s]

Processing:   0%|          | 0/26 [00:00<?, ?chunk/s]

Preprocessing:   0%|          | 0/24 [00:00<?, ?file/s]

Processing:   0%|          | 0/24 [00:00<?, ?chunk/s]

Preprocessing:   0%|          | 0/160 [00:00<?, ?file/s]

...

Processing:   0%|          | 0/25 [00:00<?, ?chunk/s]

Preprocessing:   0%|          | 0/25 [00:00<?, ?file/s]

Processing:   0%|          | 0/25 [00:00<?, ?chunk/s]

Preprocessing:   0%|          | 0/10 [00:00<?, ?file/s]

Processing:   0%|          | 0/10 [00:00<?, ?chunk/s]

In [4]:
events = loadNano("root://cmsxrootd.fnal.gov//store/group/lpcmetx/iDMe//Samples/Ntuples/signal/2018/Mchi-108p0_dMchi-36p0/ctau-1//ntuples_Mchi-108p0_dMchi-36p0_ctau-1_02.root","outT")
ele_matches = events.GenEleMatches
best_ele_match = events.GenEleMatchC
pos_matches = events.GenPosMatches
best_pos_match = events.GenPosMatchC

In [6]:
allow11 = False

numE = ak.count(ele_matches.typ,axis=1)
numP = ak.count(pos_matches.typ,axis=1)
idx = ak.Array(np.arange(len(best_ele_match)))

partitions = []
ele = []
pos = []

for i in range(1):
    # removing events where one or more objects has no match
    cutA = (numE > 0) & (numP > 0)
    ele.append(best_ele_match[~cutA])
    pos.append(best_pos_match[~cutA])
    partitions.append(idx[~cutA])
    ele_matches = ele_matches[cutA]
    best_ele_match = best_ele_match[cutA]
    pos_matches = pos_matches[cutA]
    best_pos_match = best_pos_match[cutA]
    idx = idx[cutA]
    numE = numE[cutA]
    numP = numP[cutA]

    if len(numE) == 0:
        break

    # remove events with distinct match
    cutB = (best_ele_match.typ == best_pos_match.typ) & (best_ele_match.ind == best_pos_match.ind)
    ele.append(best_ele_match[~cutB])
    pos.append(best_pos_match[~cutB])
    partitions.append(idx[~cutB])
    ele_matches = ele_matches[cutB]
    best_ele_match = best_ele_match[cutB]
    pos_matches = pos_matches[cutB]
    best_pos_match = best_pos_match[cutB]
    idx = idx[cutB]
    numE = numE[cutB]
    numP = numP[cutB]

    if len(numP) == 0:
        break

    # sorting remaining matches by dR
    sort_ele = ak.argsort(ele_matches.dr,axis=1)
    sort_pos = ak.argsort(pos_matches.dr,axis=1)
    ele_matches = ele_matches[sort_ele]
    pos_matches = pos_matches[sort_pos]

    cutC = (numE == 1) & (numP == 1)
    cutD = (numE >= 2) & (numP == 1)
    cutE = (numE == 1) & (numP >= 2)
    cutF = (numE >= 2) & (numP >= 2)

    if ak.count_nonzero(cutC) > 0:
        ele_best = best_ele_match[cutC]
        pos_best = best_pos_match[cutC]
        if not allow11:
            ele_best = ak.where(ele_best.dr < pos_best.dr,ele_best,ak.full_like(ele_best,0))
            pos_best = ak.where(pos_best.dr < ele_best.dr,pos_best,ak.full_like(pos_best,0))
        ele.append(ele_best)
        pos.append(pos_best)
        partitions.append(idx[cutC])
    if ak.count_nonzero(cutD) > 0:
        ele_best = ele_matches[cutD][:,1]
        pos_best = best_pos_match[cutD]
        ele.append(ele_best)
        pos.append(pos_best)
        partitions.append(idx[cutD])
    if ak.count_nonzero(cutE) > 0:
        ele_best = best_ele_match[cutE]
        pos_best = pos_matches[cutE][:,1]
        ele.append(ele_best)
        pos.append(pos_best)
        partitions.append(idx[cutE])
    if ak.count_nonzero(cutF) > 0:
        ele_best = ak.where(best_ele_match[cutF].dr < best_pos_match[cutF].dr,ele_matches[cutF][:,0],ele_matches[cutF][:,1])
        pos_best = ak.where(best_pos_match[cutF].dr < best_ele_match[cutF].dr,pos_matches[cutF][:,0],pos_matches[cutF][:,1])
        ele.append(ele_best)
        pos.append(pos_best)
        partitions.append(idx[cutF])

all_partitions = ak.concatenate(partitions)
reorder = ak.argsort(all_partitions)
ele_out = ak.concatenate(ele)[reorder]
pos_out = ak.concatenate(pos)[reorder]

ValueError: cannot broadcast RegularArray of size 2 with RegularArray of size 6

(https://github.com/scikit-hep/awkward-1.0/blob/1.5.0/src/awkward/_util.py#L909)

In [16]:
best_ele_match[cutF]

<NanoCollectionArray [{dr: 0.000263, ind: 0, ... typ: 1}] type='2 * struct[["dr"...'>